In [2]:
import caffe

from caffe import layers as L, params as P, to_proto

from caffe.proto import caffe_pb2

from caffe.coord_map import crop

import copy
import numpy as np

In [3]:
def conv_relu(bottom, ks, nout, stride=1, pad=0,axis=1, wf={"type":"msra"}):
    conv = L.Convolution(bottom,axis=axis, kernel_size=ks, stride=stride,
                                num_output=nout, pad=pad, weight_filler=wf)
    return L.ReLU(conv, in_place=True)

def fc_relu(bottom, nout):
    fc = L.InnerProduct(bottom, num_output=nout)
    return fc, L.ReLU(fc, in_place=True)

def max_pool(bottom, ks=2, stride=2):
    return L.Pooling(bottom, pool=P.Pooling.MAX, kernel_size=ks, stride=stride)

def deconv_relu(bottom, ks, nout, stride=1, pad=0,wf={"type":"msra"}):
    convolution_param = dict(num_output=nout, kernel_size=ks, stride=stride,pad=pad,weight_filler=wf,
            bias_term=False)
    deconv = L.Deconvolution(bottom,convolution_param=convolution_param)
    return L.ReLU(deconv, in_place=True)
    

# def three_d_conv_relu(bottom, ks, nout, stride,pad):
#     conv = L.Convolution(bottom, kernel_size=ks, stride=stride,
#                                 num_output=nout, pad=pad, group=group, weight_filler={"type":"msra"})
#     return L.ReLU(conv, in_place=True)

    
    

def get_blob_shape(blob):
    _,fn = write_to_file(blob,filename="tmp")
    net = caffe.Net(fn, caffe.TEST)
    blob_key = net.blobs.keys()[-1]
    blob_obj = net.blobs[blob_key]
    blob_shape = [blob_obj.shape[i] for i in range(len(blob_obj.shape))]
    return blob_shape
    
    
    

def write_to_file(layer,filename="train"):
    
    prototxt_str = str(to_proto(layer))

    #print prototxt_str
    with open(filename + ".prototxt", 'w') as f:
        f.write(prototxt_str)
    return prototxt_str, filename + ".prototxt"

In [9]:
SUM=1
batch_size=1

data = L.DummyData(shape={'dim':[batch_size,16,8,768,1152]}, data_filler={"type":"gaussian", "mean":0, "std":1})
label = L.DummyData(shape={'dim':[batch_size,10,12,18]}, data_filler={"type":"gaussian", "mean":0, "std":1})


nfilters_list = [128, 256, 512, 768, 1024, 1280]
nfilters_list = [f/2 for f in nfilters_list]
conv = data

num_layers = len(nfilters_list)
encoder_blobs = [data]
for i in range(num_layers):
    nfilters = nfilters_list[i]
    #print get_blob_shape(conv)
    conv = conv_relu(conv, ks=[3,5,5], stride=[1,2,2], pad=[1,2,2], nout=nfilters_list[i],
                     wf={"type":"gaussian", "mean":0, "std":1})
    encoder_blobs.append(conv)
#print get_blob_shape(conv)
nfilters_list.pop()
nfilters_list.reverse()
encoder_blobs.pop()
encoder_blobs.reverse()
nfilters_list.append(16)

encoder = conv
for i in range(num_layers):
    nfilters = nfilters_list[i]
    #print get_blob_shape(conv)
    conv = deconv_relu(conv,ks=[3,5,5], stride=[1,2,2], nout=nfilters_list[i],
                     wf={"type":"gaussian", "mean":0, "std":1})

    
    conv = L.Crop(conv,encoder_blobs[i], axis=2,offset=1)
    print get_blob_shape(conv)
#print get_blob_shape(conv)

rec = conv

conv1 = conv_relu(encoder,ks=[8,1,1],stride=[1,1,1], nout=4,wf={"type":"gaussian", "mean":0, "std":1})
print get_blob_shape(conv1)
conv2 = L.Convolution(encoder,kernel_size=[8,1,1], stride=1, num_output=6, 
                      weight_filler={"type":"gaussian", "mean":0, "std":1})
print get_blob_shape(conv2)
conv2 = L.Sigmoid(conv2)
yolo = L.Concat(conv1,conv2)
print get_blob_shape(yolo)
yolo_shape = get_blob_shape(yolo)
yolo_shape.reverse()
yolo_shape.remove(1)
yolo_shape.reverse()
print yolo_shape
yolo = L.Reshape(yolo, shape={"dim":yolo_shape})
print get_blob_shape(yolo)
loss1 = L.EuclideanLoss(yolo, label)

rec_shape = get_blob_shape(rec)
rec_shape = [rec_shape[0], rec_shape[1]*rec_shape[2], rec_shape[3], rec_shape[4]]
print rec_shape
rec = L.Reshape(rec, shape={"dim":rec_shape})

data_shape = get_blob_shape(data)
data_shape = [data_shape[0], data_shape[1]*data_shape[2], data_shape[3], data_shape[4]]
reshp_data = L.Reshape(data, shape={"dim":data_shape})
loss2 = L.EuclideanLoss(rec,reshp_data)
final_loss = L.Eltwise(loss1,loss2,operation=SUM)
pstr, fn = write_to_file(final_loss, "rect_reduce_3d")

[1, 512, 8, 24, 36]
[1, 384, 8, 48, 72]
[1, 256, 8, 96, 144]
[1, 128, 8, 192, 288]
[1, 64, 8, 384, 576]
[1, 16, 8, 768, 1152]
[1, 4, 1, 12, 18]
[1, 6, 1, 12, 18]
[1, 10, 1, 12, 18]
[1, 10, 12, 18]
[1, 10, 12, 18]
[1, 128, 768, 1152]


In [6]:
print pstr

layer {
  name: "DummyData1"
  type: "DummyData"
  top: "DummyData1"
  dummy_data_param {
    data_filler {
      type: "gaussian"
      mean: 0
      std: 1
    }
    shape {
      dim: 1
      dim: 16
      dim: 8
      dim: 768
      dim: 1152
    }
  }
}
layer {
  name: "Convolution1"
  type: "Convolution"
  bottom: "DummyData1"
  top: "Convolution1"
  convolution_param {
    num_output: 64
    pad: 1
    pad: 2
    pad: 2
    kernel_size: 3
    kernel_size: 5
    kernel_size: 5
    stride: 1
    stride: 2
    stride: 2
    weight_filler {
      type: "gaussian"
      mean: 0
      std: 1
    }
    axis: 1
  }
}
layer {
  name: "ReLU1"
  type: "ReLU"
  bottom: "Convolution1"
  top: "Convolution1"
}
layer {
  name: "Convolution2"
  type: "Convolution"
  bottom: "Convolution1"
  top: "Convolution2"
  convolution_param {
    num_output: 128
    pad: 1
    pad: 2
    pad: 2
    kernel_size: 3
    kernel_size: 5
    kernel_size: 5
    stride: 1
    stride: 2
    stride: 2
    weight_fil

In [5]:
net=caffe.Net(fn,caffe.TRAIN)

In [ ]:
c1 = net.forward()

In [ ]:
! cat 